## Questions

### 1 Meanshift
q1. Experiment the tracking performed by the provided code Tracking_MeanShift.py that uses the
basic Mean Shift algorithm, on the marginal density of the hue comp onent H. Explain the principle
of this algorithm, and illustrate its advantages and limits by your exp eriments.

In [1]:

import sys
sys.path.append('..')
from src.classical_tracker import ClassicalTracker

VIDEO_PATH = '../Test-Videos/Antoine_Mug.mp4'

In [ ]:
tracker = ClassicalTracker(
    video_path=VIDEO_PATH,
    method='meanshift',
    color_space='hue'
)
tracker.track_video(visualize=True, save_result=True, output_dir='../results/q1_basic')

Step 1: Select ROI
Instructions:
- Drag mouse to select ROI
- Press 'q' to confirm and start tracking
Step 2: Initialize tracker
Mean Shift initialized with hue histogram
Step 3: Start tracking
Press 's' to save frame, 'ESC' to exit


: 